#### Predict TTennis data
1. Load TTShort.csv
2. (done in pre) Load new games, format to TTShort
3. (done in pre) Append new games to TTShort.csv
4. Run sliding window logic to populate missing data
5. 

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

#### Load TTShort.csv

In [2]:
# dfUpcoming = pd.read_csv("./ttennisData/TTCup.csv")
df = pd.read_csv("/Users/forbesjon2/Desktop/SportsBet/mlb/MLB-main/TableTennis/TTCup.csv")
df = df.fillna(0) # G4+ have 


# ------- Create new columns for df -------
# Combine P1 and P2 scores for each game
for i in range(1, 6):
    df[f'G{i}_Total'] = df[f'P1_G{i}'] + df[f'P2_G{i}']

# Player 1 total score from G1 to G5
df['Total_P1'] = df[[f'P1_G{i}' for i in range(1, 6)]].sum(axis=1)

# Player 2 total score from G1 to G5
df['Total_P2'] = df[[f'P2_G{i}' for i in range(1, 6)]].sum(axis=1)

# Sum of combined scores from G1 to G5
df['Total_Score'] = df[[f'G{i}_Total' for i in range(1, 6)]].sum(axis=1)
df['Over_74'] = df['Total_Score'] > 74

df.rename(columns={'P1_Total': 'Total_P1', 'P2_Total': 'Total_P2'}, inplace=True)

# df['No_Odds'] = (df['Odds_P1'] + df['Odds_P2'] == 0)
# print(df['No_Odds'].sum())

df['Total_Allowed_P1'] = df['Total_P2']
df['Total_Allowed_P2'] = df['Total_P1']

df = df.sort_values(by='Date')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110420 entries, 110389 to 110419
Data columns (total 31 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              110420 non-null  float64
 1   Player1           110420 non-null  float64
 2   Player2           110420 non-null  float64
 3   Player1_Name      110420 non-null  object 
 4   Player2_Name      110420 non-null  object 
 5   Sets_P1           110420 non-null  float64
 6   Sets_P2           110420 non-null  float64
 7   Odds_P1           110420 non-null  float64
 8   Odds_P2           110420 non-null  float64
 9   P1_G1             110420 non-null  float64
 10  P2_G1             110420 non-null  float64
 11  P1_G2             110420 non-null  float64
 12  P2_G2             110420 non-null  float64
 13  P1_G3             110420 non-null  float64
 14  P2_G3             110420 non-null  float64
 15  P1_G4             110420 non-null  float64
 16  P2_G4             11

/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_1891/2863649388.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/forbesjon2/Desktop/SportsBet/mlb/MLB-main/TableTennis/TTCup.csv")


In [3]:
cont_cols = [
    'Date',
    'Player1',
    'Player2',
    'Sets_P1',
    'Sets_P2',
#    'No_Odds',
    'Total_P1',
    'Total_P2',
    'Total_Score',
    'Total_Allowed_P1',
    'Total_Allowed_P2',
]

track_cols = [
    'Date',
    'Sets',
    'Odds',
    'Total',
    'Total_Allowed',
]

y_col = ['H_won']

# Convert columns to int (it originally was int before odds data was added)
for col in cont_cols:
    df[col] = df[col].astype(np.int64)
cont_cols = cont_cols + ['Over_74','Odds_P1','Odds_P2','Player1_Name', 'Player2_Name']

dfOriginal = df
df = df[cont_cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110420 entries, 110389 to 110419
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              110420 non-null  int64  
 1   Player1           110420 non-null  int64  
 2   Player2           110420 non-null  int64  
 3   Sets_P1           110420 non-null  int64  
 4   Sets_P2           110420 non-null  int64  
 5   Total_P1          110420 non-null  int64  
 6   Total_P2          110420 non-null  int64  
 7   Total_Score       110420 non-null  int64  
 8   Total_Allowed_P1  110420 non-null  int64  
 9   Total_Allowed_P2  110420 non-null  int64  
 10  Over_74           110420 non-null  bool   
 11  Odds_P1           110420 non-null  float64
 12  Odds_P2           110420 non-null  float64
 13  Player1_Name      110420 non-null  object 
 14  Player2_Name      110420 non-null  object 
dtypes: bool(1), float64(2), int64(10), object(2)
memory usage: 12.7+ MB


### 2. Create a dict to track the track_cols array
create another dict to track previous games for each team during the year


In [4]:
df.head()

,Date,Player1,Player2,Sets_P1,Sets_P2,Total_P1,Total_P2,Total_Score,Total_Allowed_P1,Total_Allowed_P2,Over_74,Odds_P1,Odds_P2,Player1_Name,Player2_Name
110389,1612281300,329135,360367,2,3,46,50,96,50,46,True,1.27,3.35,0,0
110388,1612283400,352537,328779,1,3,34,38,72,38,34,False,2.10,1.65,0,0
110387,1612285500,329135,352537,3,0,33,14,47,14,33,False,1.60,2.20,0,0
110386,1612287600,376223,360367,0,3,14,33,47,33,14,False,1.85,1.85,0,0
110385,1612289700,329135,376223,1,3,32,42,74,42,32,False,1.35,2.95,0,0


### Create dfUpcoming with upcoming records, df with past games

In [5]:
# Copy of df without upcoming data (use the sum of last 10 columns as a judge)
dfUpcoming = df
last10 = dfUpcoming.iloc[:,-8:-5]
rowSum = last10.sum(axis=1)
df = dfUpcoming[rowSum != 0]


print(df.shape)
print(dfUpcoming.shape)


(110394, 15)
(110420, 15)


In [6]:
track_dict = {}

for row in df.itertuples():
    year = datetime.fromtimestamp(row.Date).year # row.Date.split('-')[0]
    # year = row.Season
    home_team = row.Player1
    visitor_team = row.Player2

    
    # Home or visitor team has < minimum_window total games
    for col in track_cols:
        home_column_name = f'{year}_{home_team}_{col}'
        visitor_column_name = f'{year}_{visitor_team}_{col}'
        
        # Home team
        home_col = col if col == 'Date' else col + '_P1'
        if home_column_name in track_dict:
            track_dict[home_column_name].append(getattr(row, home_col))
        else:
            track_dict[home_column_name] = [getattr(row, home_col)]
        
        # Visitor team
        visitor_col = col if col == 'Date' else col + '_P2'
        if visitor_column_name in track_dict:
            track_dict[visitor_column_name].append(getattr(row, visitor_col))
        else:
            track_dict[visitor_column_name] = [getattr(row, visitor_col)]

In [7]:
print(df.shape)
print(dfUpcoming.shape)
onlyUpcoming = dfUpcoming.merge(df, how='outer', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
print(onlyUpcoming.shape)


(110394, 15)
(110420, 15)
(26, 15)


### Check and remove missing data

In [8]:
df = dfUpcoming
df = onlyUpcoming

minimum_window = 50
ema_span = 50
print(df.shape)

show_examples = False

indices_to_drop = []
current_count = 0

for row in df.itertuples():
    if current_count % 300 == 0:
        print(f'{current_count}/{df.shape[0]}')
    current_count = current_count + 1
    index = row.Index
    year = datetime.fromtimestamp(row.Date).year # row.Date.split('-')[0]
    # year = row.Season
    home_team = row.Player1
    visitor_team = row.Player2
    # Home team min window
    home_date_column = f'{year}_{home_team}_Date'
    visitor_date_column = f'{year}_{visitor_team}_Date'

    # Drop indice if not enough data
    if len(track_dict[home_date_column]) <= minimum_window or len(track_dict[visitor_date_column]) <= minimum_window:
        indices_to_drop.append(index)
        continue

    # Current row is older than Home team at min_window
    if row.Date <= track_dict[home_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue
    # Current row is older than Visitor team at min_window
    if row.Date <= track_dict[visitor_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue

    # 9/15/25 - Future games have no index, get last index if not found.
    # home_date_index = track_dict[home_date_column].index(row.Date)
    # home_date_index = track_dict[home_date_column].index(row.Date) if row.Date in track_dict[home_date_column] else len(track_dict[home_date_column]) - 1
    home_date_index = len(track_dict[home_date_column]) - 1

    # visitor_date_index = track_dict[visitor_date_column].index(row.Date)
    # visitor_date_index = track_dict[visitor_date_column].index(row.Date) if row.Date in track_dict[visitor_date_column] else len(track_dict[visitor_date_column]) - 1
    visitor_date_index = len(track_dict[visitor_date_column]) - 1
    print(f'H: {home_date_index} V: {visitor_date_index}')

    # print("----- Track Cols -----")
    # Update df to have average for each track_cols (Ignoring 'Date', the 1-2nd item)
    for col in track_cols:
        if col in ['Date', 'Odds']:
            continue
        else:
            # print(col)
            # Convert to float
            if df.dtypes[col + '_P1'] == 'int64' or df.dtypes[col + '_P2'] == 'int64':
                df = df.astype({f'{col}_P1': 'float64', f'{col}_P2': 'float64'})
            # Update df to have average for each track_cols (Ignoring 'Date', .. items)
            # Update home
            home_col_list = track_dict[f'{year}_{home_team}_{col}'][:home_date_index-1]
            dataframe_val = pd.DataFrame({'value': home_col_list})
            ema = dataframe_val['value'].ewm(span=min(ema_span, len(home_col_list)), adjust=True).mean().iloc[-1]
            df.at[index, col + '_P1'] = ema

            # Update Visitor
            visitor_col_list = track_dict[f'{year}_{visitor_team}_{col}'][:visitor_date_index-1]
            dataframe_val = pd.DataFrame({'value': visitor_col_list})
            ema = dataframe_val['value'].ewm(span=min(ema_span, len(visitor_col_list)), adjust=True).mean().iloc[-1]
            df.at[index, col + '_P2'] = ema


    # --------------------------------------------------- 
    # ------------------ Custom Columns -----------------
    # --------------------------------------------------- 
    #
    
    # 1. Add variant of Bill James pythagorean expectation (NFL).
    # Recent games weighted more heavily since 'Final' columns not excluded from the above loop
    home_points_for = sum(track_dict[f'{year}_{home_team}_Total'][:home_date_index-1])
    home_points_against = sum(track_dict[f'{year}_{home_team}_Total_Allowed'][:home_date_index-1])
    df.at[index, 'pythagorean_P1'] = home_points_for**1.83 / (home_points_for**1.83 + home_points_against**1.83)

    visitor_points_for = sum(track_dict[f'{year}_{visitor_team}_Total'][:visitor_date_index-1])
    visitor_points_against = sum(track_dict[f'{year}_{visitor_team}_Total_Allowed'][:visitor_date_index-1])
    df.at[index, 'pythagorean_P2'] = visitor_points_for**1.83 / (visitor_points_for**1.83 + visitor_points_against**1.83)
        
df.drop(indices_to_drop, inplace=True)

# Add custom metrics to track_cols so it creates the difference (D_) column
# track_cols.append('pythagorean')
for col in track_cols[1:]:
    cont_cols.append('D_' + col)
    df['D_' + col] = (df[col + '_P1'] - df[col + '_P2']).round(3) # Round to 3 sig figs
    
# Remove columns that begin with H_ or V_ in df
y_col = ['H_Won', 'H_start_odds', 'V_start_odds']
df = df.loc[:, ~(df.columns.str.startswith(('H_', 'V_')) & (~df.columns.isin(y_col)))]

print(df.shape)


(26, 15)
0/26
H: 949 V: 951
H: 87 V: 89
H: 103 V: 55
H: 87 V: 55
H: 1008 V: 624
H: 89 V: 103
H: 87 V: 103
H: 55 V: 89
H: 406 V: 460
H: 533 V: 582
H: 380 V: 482
H: 84 V: 475
H: 78 V: 196
H: 133 V: 380
H: 84 V: 196
H: 492 V: 243
H: 475 V: 78
H: 133 V: 243
H: 84 V: 78
H: 380 V: 492
H: 196 V: 475
H: 133 V: 492
H: 343 V: 780
H: 582 V: 356
H: 419 V: 188
H: 487 V: 380
(26, 21)


In [9]:
df.info()
df.to_csv(f'./ttennisData/TTCupSlidingTESTUPC.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 110636 to 110665
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              26 non-null     int64  
 1   Player1           26 non-null     int64  
 2   Player2           26 non-null     int64  
 3   Sets_P1           26 non-null     float64
 4   Sets_P2           26 non-null     float64
 5   Total_P1          26 non-null     float64
 6   Total_P2          26 non-null     float64
 7   Total_Score       26 non-null     int64  
 8   Total_Allowed_P1  26 non-null     float64
 9   Total_Allowed_P2  26 non-null     float64
 10  Over_74           26 non-null     bool   
 11  Odds_P1           26 non-null     float64
 12  Odds_P2           26 non-null     float64
 13  Player1_Name      26 non-null     object 
 14  Player2_Name      26 non-null     object 
 15  pythagorean_P1    26 non-null     float64
 16  pythagorean_P2    26 non-null     float64


### Import & apply model
1. Import the saved classifier, calibration model
2. Apply the model to the upcoming data and filter lower/upper bound


In [10]:
x_col = [
    'Sets_P1',
    'Sets_P2',
    'Total_P1',
    'Total_P2',
    'Total_Allowed_P1',
    'Total_Allowed_P2',
    'pythagorean_P1',
    'pythagorean_P2',
    'D_Sets',
    'D_Odds',
#    'No_Odds',
    'D_Total',
    'D_Total_Allowed'
]
identifier_col = ['Date', 'Player1_Name', 'Player2_Name', 'Player1', 'Player2']

# Get upcoming data
dfUpcoming = df[df['Total_Score'] == 0]
print(dfUpcoming.shape)

xId = dfUpcoming[identifier_col]
x = dfUpcoming[x_col]
print(x.shape)

conts = np.stack([x[col].values for col in list(x.columns)], 1)
idConts = np.stack([xId[col].values for col in list(xId.columns)], 1)

(26, 21)
(26, 12)


In [14]:
import pickle

# Import saved classifier, calibrator (will error out if DNE)
with open('ttennis_classifier.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('ttennis_calibrator.pkl', 'rb') as f:
    calibrator = pickle.load(f)


# Apply the model to the upcoming data
perf_conts_res = classifier.predict_proba(conts)[:,1]
perf_conts_calibrated = calibrator.predict(perf_conts_res)
perf_conts_calibrated_2d = np.stack([
    1 - perf_conts_calibrated,  # Index 0: P(class=0)
    perf_conts_calibrated       # Index 1: P(class=1)
], axis=1)
perf_conts_calibrated = perf_conts_calibrated_2d
positive_probs = perf_conts_calibrated[:, 1]

# Define bounds around 0.5
limit = 0.07
lower_bound = 0.5 - limit
upper_bound = 0.5 + limit

# Filter lower/upper bound
mask = (positive_probs < lower_bound) | (positive_probs > upper_bound)
perf_conts_calibrated = perf_conts_calibrated[mask]
conts_id = idConts[mask]

#print(perf_conts_calibrated.shape)
#print(conts_id.shape)
#print(conts_id)

In [15]:
# print(perf_conts_calibrated)
for i in range(0,len(perf_conts_calibrated)):
    over_745 = perf_conts_calibrated[i][1] > perf_conts_calibrated[i][0]

    # Create searchString
    dateFloat = float(conts_id[i][0])
    playerOneFloat = float(conts_id[i][3])
    playerTwoFloat = float(conts_id[i][4])
    searchString = f"{dateFloat},{playerOneFloat},{playerTwoFloat}"

    # Get timestamp, convert from UTC to EST
    utc_dt = datetime.utcfromtimestamp(int(conts_id[i][0]))
    edt_dt = utc_dt - timedelta(hours=4)  # adjust for EDT
    dts = edt_dt.strftime('%Y-%m-%d %H:%M:%S')
    
    print(f"Date: {dts}, Over 74.5: {over_745}, {conts_id[i][1]} vs {conts_id[i][2]}, SearchString: {searchString}")

Date: 2025-09-21 15:10:00, Over 74.5: True, Blazej Warpas vs Michal Lysakowski, SearchString: 1758481800.0,707970.0,311807.0


/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_1891/1890001091.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_dt = datetime.utcfromtimestamp(int(conts_id[i][0]))
